# 1. What interesting information user can find in the dataset?

###### Input data summary

 A. 109 whiskies are described by 5 characteristics: color (14 options),nose (12 options),body (8 options), palate (15 options), finish (19 options). For each characteristic, 1 whisky can have only 1 option selected.

 B. for each whisky there is distillery assigned. Distillery might be identified by geographical coordinates (latitude/longitude) and region/district (which indicates macrogeographic location, such as proximity to the sea, altiude etc.)

 C. for each whisky there is score indicating quality (varies from 55% to 90%): score is based on one of references invoked in "Lapointe, F.-J. & P. Legendre. 1994. A classification of pure malt Scotch whiskies." study, which is starting point for this study.

###### Identified potential data applications


 A. relation between whisky quality score and different characteristics: check if whiskies scoring higher are more likely to have certain values of color,nose, body, palate and finish ?

 B. relation between distillery location/district/region and whisky's characteristics: is there any ? (for example, if distilleries of lower altitude tend to produce whisky of certain colour ?)

 C. similarity of whiskies (being analazyed in further part if this study): considering characteristics for each whisky, how "similar" they are to each other ?

# 2. Core code: determine whiskies similarity

###### Define function for whisky similarity, including only one characteristic

In [54]:
import pandas as pd

# define function that returns column of "similarities" - 1 means whiskyID is similar
# in terms of considered characteristic to the given row

def char_similarity(charFilename,whiskyID):
    df = pd.read_csv(charFilename,delimiter = " ", header = None, skiprows=1)# read data for specified characteristics
    df2 = df.subtract(df.iloc[whiskyID],axis=1)# substract row of interest from every other row
    df2['similarity'] = 1 - df2.max(axis=1) # if whisky was similar to given row, whole row is 0
    return df2['similarity']

print(char_similarity('COLOR.TXT',1)) # EXAMPLE: test color similarity of whisky indexed as 1 to all whiskies

0      0
1      1
2      0
3      0
4      0
      ..
104    0
105    0
106    0
107    0
108    0
Name: similarity, Length: 109, dtype: int64


###### Define function that will calculate similarity of one whisky to another, including all characteristic.

ASSUMPTION: all characteristic are considered to have even weight

In [55]:
def whiskies_similarity(charList,whiskyID):
    i=0
    for characteristic in charList:
        if i==0:
            df = pd.DataFrame(char_similarity(characteristic,whiskyID))# read/process first characteristic
            i = 1
        else:
            df = df.join(char_similarity(characteristic,whiskyID),rsuffix=charList[i]) #read/process further characteristics
            i=i+1
    df.columns=charList # rename columns to match characteristics
    df['similarity'] = df.mean(axis=1)
    df['name'] = pd.read_csv("NAMES_PSutkowski.txt",header=None) # read whiskies names from custo
    return df
charList = ['COLOR.TXT','NOSE.TXT','BODY.TXT','PALATE.TXT','FINISH.TXT'] 

print(whiskies_similarity(charList,77).sort_values(by='similarity',ascending=False))
    

     COLOR.TXT  NOSE.TXT  BODY.TXT  PALATE.TXT  FINISH.TXT  similarity  \
77           1       1.0       1.0           1           1         1.0   
2            0       1.0       0.0           0           1         0.4   
3            0       0.0       1.0           0           1         0.4   
95           0       1.0       0.0           0           1         0.4   
23           0       1.0       1.0           0           0         0.4   
..         ...       ...       ...         ...         ...         ...   
37           0       0.0       0.0           0           0         0.0   
36           0       0.0       0.0           0           0         0.0   
35           0       0.0       0.0           0           0         0.0   
34           0       0.0       0.0           0           0         0.0   
108          0       0.0       0.0           0           0         0.0   

           name  
77    Lagavulin  
2       Ardberg  
3       Ardmore  
95    Singleton  
23    Clynelish  
.. 

# 3. Data visualization and further processing